# Pentathlon-III: Next Product to Buy Models

* Team-lead GitLab userid:
* Group name:
* Team member names:

In [1]:
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyrsm as rsm
import statsmodels.formula.api as smf
from sklearn import metrics, preprocessing
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.families.links import logit

warnings.filterwarnings("ignore")
# increase plot resolution
# mpl.rcParams["figure.dpi"] = 150

In [2]:
## loading the data - this dataset must NOT be changed
pentathlon_nptb = pd.read_pickle("data/pentathlon_nptb.pkl")
pentathlon_nptb["buyer_yes"] = (pentathlon_nptb["buyer"] == "yes").astype(int)

In [3]:
pent = pentathlon_nptb

In [4]:
## Question answers

# Neural Network

#### For our neural network model, we created dummy variables for the categorical variables, age, gender, and message.

In [5]:
# Dummy variables created for gender
pent["gender"] = pd.get_dummies(pent["gender"])
pent[
    ["under_thirty", "thirty_to_fortyfour", "fortyfive_to_fiftynine", "sixty_and_above"]
] = pd.get_dummies(pent["age"])

In [6]:
rvar = "buyer_yes"
evar = [
    "income",
    "education",
    "children",
    "gender",
    "under_thirty",
    "thirty_to_fortyfour",
    "fortyfive_to_fiftynine",
    "sixty_and_above",
    "freq_endurance",
    "freq_strength",
    "freq_water",
    "freq_team",
    "freq_backcountry",
    "freq_winter",
    "freq_racquet",
    "message_endurance",
    "message_strength",
    "message_water",
    "message_team",
    "message_backcountry",
    "message_winter",
    "message_racquet",
]
std_var = [
    "education",
    "children",
    "income",
    "freq_endurance",
    "freq_strength",
    "freq_water",
    "freq_team",
    "freq_backcountry",
    "freq_winter",
    "freq_racquet",
]

idvar = "custid"
lev = 1

In [7]:
pd.set_option("display.max_columns", None)
# standardization-- when standardizing, do we look at training == 1 or do we look at representative == 1?
X_colnames = pent.loc[
    [0],
    [
        "education",
        "children",
        "income",
        "freq_endurance",
        "freq_strength",
        "freq_water",
        "freq_team",
        "freq_backcountry",
        "freq_winter",
        "freq_racquet",
    ],
].columns
scaler = preprocessing.StandardScaler()
sf = scaler.fit(pent.query("training == 1")[X_colnames])
pent_std = pent.copy()

# new standardized cols
pent_std[std_var] = sf.transform(pent[X_colnames])

In [9]:
# Dummy variables created for message
pq = pd.get_dummies(pent_std[["message"]])
pent_std = pent_std.join(pq)
train = pent_std[pent_std["training"] == 1]
test = pent_std[pent_std["training"] == 0]
rep = pent_std[pent_std["representative"] == 1]

In [10]:
# Training and testing sets created for training and representative
eval_dat = pd.concat([train, test, rep], axis=0)
eval_dat = eval_dat[[idvar, rvar, "training", "representative"]]
X_train = train[evar]
y_train = train[rvar]
X_test = test[evar]
y_test = test[rvar]
X_rep = rep[evar]
y_rep = rep[rvar]

In [11]:
# # Full sets created for training and representative
# X_full = pent_std[evar]
# y_full = pent_std[rvar]

In [12]:
# Neural net from SKLEARN
clf = MLPClassifier(
    activation="relu",
    solver="adam",
    alpha=0.45,
    hidden_layer_sizes=(4,),
    random_state=1234,
    max_iter=10000,
).fit(X_train, y_train)

In [13]:
dfs = [X_train, X_test]
Xs = pd.concat(dfs).reset_index(drop=True)

In [35]:
pred_2_tr = clf.predict_proba(X_train)
fpr, tpr, thresholds = metrics.roc_curve(y_train, pred_2_tr[:, 1])

pred_2_test = clf.predict_proba(X_test)
fpr_t, tpr_t, thresholds_t = metrics.roc_curve(y_test, pred_2_test[:, 1])

print(f"AUC training data for neural network model: {(metrics.auc(fpr, tpr) * 100).round(2)}%")
print(f"AUC testing data for neural network model: {(metrics.auc(fpr_t, tpr_t) * 100).round(2)}%\n")

AUC training data for neural network model: 88.46%
AUC testing data for neural network model: 88.45%



The cross validation/GridSearchCV below is commented out to help performance times.

In [15]:
# # CV for NN
# nr_hnodes = range(2, 5)
# hls = list(zip(nr_hnodes)) + list(zip(nr_hnodes, nr_hnodes))

In [16]:
# # alpha is the level of regularization
# param_grid = {
#     "hidden_layer_sizes": hls,
#     "alpha": [0.4, 0.45, 0.5],
#     "random_state": [1234],
# }
# scoring = {"AUC": "roc_auc"}

# clf_cv = GridSearchCV(
#     clf,
#     param_grid,
#     scoring=scoring,
#     cv=5,
#     n_jobs=4,
#     refit="AUC",
#     verbose=5,
# )
# clf_cv.fit(X_train, y_train)

In [17]:
# Predictions on final model
# strength
strength_test = X_test.copy()
strength_train = X_train.copy()
# strength_test["message_strength"]=1
# strength_test[["message_endurance",	"message_water", "message_team","message_backcountry","message_winter","message_racquet"]]=0
# strength_train["message_strength"]=1
# strength_train[["message_endurance", "message_water", "message_team","message_backcountry","message_winter","message_racquet"]]=0
# strength_test
strength_rep = X_rep.copy()
strength_rep["message_strength"] = 1
strength_rep[
    [
        "message_racquet",
        "message_water",
        "message_team",
        "message_backcountry",
        "message_winter",
        "message_endurance",
    ]
] = 0

eval_dat["strength_p"] = 0
eval_dat.loc[eval_dat.representative == 1, "strength_p"] = clf.predict_proba(
    strength_rep
)[:, 1]
# eval_dat.loc[eval_dat.training == 1, "strength_p"] = clf.predict_proba(strength_train)[:, 1]
# eval_dat.loc[eval_dat.training == 0, "strength_p"] = clf.predict_proba(strength_test)[:, 1]

In [18]:
# Predictions on final model
# strength
strength_test = X_test.copy()
strength_train = X_train.copy()
# strength_test["message_endurance"]=1
# strength_test[["message_strength",	"message_water", "message_team","message_backcountry","message_winter","message_racquet"]]=0
# strength_train["message_endurance"]=1
# strength_train[["message_strength", "message_water", "message_team","message_backcountry","message_winter","message_racquet"]]=0
# strength_test
strength_rep = X_rep.copy()
strength_rep["message_endurance"] = 1
strength_rep[
    [
        "message_strength",
        "message_water",
        "message_team",
        "message_backcountry",
        "message_winter",
        "message_racquet",
    ]
] = 0

eval_dat["endurance_p"] = 0
eval_dat.loc[eval_dat.representative == 1, "endurace_p"] = clf.predict_proba(
    strength_rep
)[:, 1]
# eval_dat.loc[eval_dat.training == 1, "endurance_p"] = clf.predict_proba(strength_train)[:, 1]
# eval_dat.loc[eval_dat.training == 0, "endurance_p"] = clf.predict_proba(strength_test)[:, 1]

In [19]:
# Predictions on final model
# strength
strength_test = X_test.copy()
strength_train = X_train.copy()
# strength_test["message_water"]=1
# strength_test[["message_strength",	"message_endurance", "message_team","message_backcountry","message_winter","message_racquet"]]=0
# strength_train["message_water"]=1
# strength_train[["message_strength", "message_endurance", "message_team","message_backcountry","message_winter","message_racquet"]]=0
# strength_test
strength_rep = X_rep.copy()
# print(strength_rep.shape)
strength_rep["message_water"] = 1
strength_rep[
    [
        "message_strength",
        "message_endurance",
        "message_team",
        "message_backcountry",
        "message_winter",
        "message_racquet",
    ]
] = 0

eval_dat["water_p"] = 0
eval_dat.loc[eval_dat.representative == 1, "water_p"] = clf.predict_proba(strength_rep)[
    :, 1
]
# eval_dat.loc[eval_dat.training == 1, "water_p"] = clf.predict_proba(strength_train)[:, 1]
# eval_dat.loc[eval_dat.training == 0, "water_p"] = clf.predict_proba(strength_test)[:, 1]

In [20]:
# Predictions on final model
# strength
strength_test = X_test.copy()
strength_train = X_train.copy()
# strength_test["message_team"]=1
# strength_test[["message_strength",	"message_endurance", "message_water","message_backcountry","message_winter","message_racquet"]]=0
# strength_train["message_team"]=1
# strength_train[["message_strength", "message_endurance", "message_water","message_backcountry","message_winter","message_racquet"]]=0
# strength_test
strength_rep = X_rep.copy()
strength_rep["message_team"] = 1
strength_rep[
    [
        "message_strength",
        "message_water",
        "message_racquet",
        "message_backcountry",
        "message_winter",
        "message_endurance",
    ]
] = 0

eval_dat["team_p"] = 0
eval_dat.loc[eval_dat.representative == 1, "team_p"] = clf.predict_proba(strength_rep)[
    :, 1
]
# eval_dat.loc[eval_dat.training == 1, "team_p"] = clf.predict_proba(strength_train)[:, 1]
# eval_dat.loc[eval_dat.training == 0, "team_p"] = clf.predict_proba(strength_test)[:, 1]

In [21]:
# Predictions on final model
# strength
strength_test = X_test.copy()
strength_train = X_train.copy()
# strength_test["message_racquet"]=1
# strength_test[["message_strength",	"message_water", "message_team","message_backcountry","message_winter","message_endurance"]]=0
# strength_train["message_racquet"]=1
# strength_train[["message_strength", "message_water", "message_team","message_backcountry","message_winter","message_endurance"]]=0
# strength_test
strength_rep = X_rep.copy()
strength_rep["message_racquet"] = 1
strength_rep[
    [
        "message_strength",
        "message_water",
        "message_team",
        "message_backcountry",
        "message_winter",
        "message_endurance",
    ]
] = 0

eval_dat["racquet_p"] = 0
eval_dat.loc[eval_dat.representative == 1, "racquet_p"] = clf.predict_proba(
    strength_rep
)[:, 1]
# eval_dat.loc[eval_dat.training == 1, "racquet_p"] = clf.predict_proba(strength_train)[:, 1]
# eval_dat.loc[eval_dat.training == 0, "racquet_p"] = clf.predict_proba(strength_test)[:, 1]

In [22]:
# Predictions on final model
# strength
strength_test = X_test.copy()
strength_train = X_train.copy()
# strength_test["message_winter"]=1
# strength_test[["message_strength",	"message_water", "message_team","message_backcountry","message_racquet","message_endurance"]]=0
# strength_train["message_winter"]=1
# strength_train[["message_strength", "message_water", "message_team","message_backcountry","message_racquet","message_endurance"]]=0
# strength_test
strength_rep = X_rep.copy()
strength_rep["message_winter"] = 1
strength_rep[
    [
        "message_strength",
        "message_water",
        "message_team",
        "message_backcountry",
        "message_racquet",
        "message_endurance",
    ]
] = 0

eval_dat["winter_p"] = 0
eval_dat.loc[eval_dat.representative == 1, "winter_p"] = clf.predict_proba(
    strength_rep
)[:, 1]
# eval_dat.loc[eval_dat.training == 1, "winter_p"] = clf.predict_proba(strength_train)[:, 1]
# eval_dat.loc[eval_dat.training == 0, "winter_p"] = clf.predict_proba(strength_test)[:, 1]

In [23]:
# Predictions on final model
# strength
strength_test = X_test.copy()
strength_train = X_train.copy()
strength_rep = X_rep.copy()

# strength_test["message_backcountry"]=1
# strength_test[["message_strength",	"message_water", "message_team","message_winter","message_racquet","message_endurance"]]=0
# strength_train["message_backcountry"]=1
# strength_train[["message_strength", "message_water", "message_team","message_winter","message_racquet","message_endurance"]]=0
strength_rep["message_backcountry"] = 1
strength_rep[
    [
        "message_strength",
        "message_water",
        "message_team",
        "message_winter",
        "message_racquet",
        "message_endurance",
    ]
] = 0

eval_dat["backcountry_p"] = 0
# eval_dat.loc[eval_dat.training == 1, "backcountry_p"] = clf.predict_proba(strength_train)[:, 1]
# eval_dat.loc[eval_dat.training == 0, "backcountry_p"] = clf.predict_proba(strength_test)[:, 1]
eval_dat.loc[eval_dat.representative == 1, "backcountry_p"] = clf.predict_proba(
    strength_rep
)[:, 1]

In [24]:
# get the message that will result in the highest profit for each customer
eval_dat["to_offer_i"] = (
    eval_dat[
        [
            "strength_p",
            "water_p",
            "team_p",
            "backcountry_p",
            "winter_p",
            "racquet_p",
            "endurance_p",
        ]
    ]
    .idxmax(axis=1)
    .str.replace("strength_p", "strength")
    .replace("water_p", "water")
    .replace("team_p", "team")
    .replace("backcountry_p", "backcountry")
    .replace("winter_p", "winter")
    .replace("racquet_p", "racquet")
    .replace("endurance_p", "endurance")
)

In [25]:
# using the predctions for each of the messages, get the prediction of the value
eval_dat["p_target_i"] = eval_dat[
    [
        "strength_p",
        "water_p",
        "team_p",
        "backcountry_p",
        "winter_p",
        "racquet_p",
        "endurance_p",
    ]
].max(axis=1)

In [33]:
# for the entire 200K customers, get the percentage of customers for whom each message type will lead to the highest probability of purchase
full_perc = eval_dat[
    [
        "strength_p",
        "water_p",
        "team_p",
        "backcountry_p",
        "winter_p",
        "racquet_p",
        "endurance_p",
        "p_target_i",
    ]
].agg("mean").sort_values(ascending=True).apply(rsm.format_nr, perc=True)

### Q1: Determine the message (i.e., endurance, strength, water, team, backcountry, winter, or racquet) predicted to lead to the highest probability of purchase. Describe your approach.

In [29]:
# df to deal just with the representative data for questions 1-8
reps = eval_dat[eval_dat["representative"] == 1].reset_index(drop=True)

In [30]:
# For each customer, the message that will lead to the highest probability of purchase
reps[["custid", "to_offer_i"]]

,custid,to_offer_i
0,U45198803,strength
1,U22197752,strength
2,U19423462,strength
3,U23888305,strength
4,U16954857,strength
...,...,...
99995,U12620333,strength
99996,U18623424,strength
99997,U64468968,strength
99998,U33721691,strength


With the neural network, we predicted that strength was the message that would lead us to the highest probability for every customer. We also ran the model on the 5M dataset and the model led us to predict similar results.This obviously is unrealistic and we determined that the neural network would not be an accurate model for us to predict which message would lead to the highest probability of purchase. Our approach was that, in order to get reasonable predictions, without them all being equal for each message variable, we would have to set each message dummy variable to 1 and the rest to zero, making 7 predictions in total for each (this has been commented out in the code blocks above). This yielded us poor results, so we made another attempt at making predictions on the representative data, while fitting the model on the training and testing data. This also yielded us similar results with one message type dominating across the board.

### Q2: For each message, report the percentage of customers for whom that message maximizes their probability of purchase.

In [31]:
# For each of the 7 message types, plus targeting apprach, the percentage of customers for whom that message maximizes their probability of purchase
nn_q2 = reps['to_offer_i'].value_counts(normalize = True)*100
nn_q2 = pd.DataFrame(nn_q2)
nn_q2.columns = ['customers (%)']

In [32]:
nn_q2

,customers (%)
strength,100.0


For each message type, our model predicts that strength will maximize 99.99% of our customers probability of purchase for the representative data. Applying this model to the 5M dataset also yielded us with similar results, with one message dominating the probability of purchase over the rest.

At this point, because of the results of the previous two questions, we decided not to continue on with using this model to answer the next questions and focus our efforts on fine-tuning model types that will provide us with better predictions. The neural network did not give us the predictions we were hoping for, likely because of the type of data at hand.